In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection


c:/Users/PC/Data/ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [2]:
save_dir=f"{exps_dir}/exp_prj_credit_card_transactions_fraud"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
x_train=pd.read_csv(f'{save_dir}/x_train.csv')
y_train=pd.read_csv(f'{save_dir}/y_train.csv')
x_test=pd.read_csv(f'{save_dir}/x_test.csv')
y_test=pd.read_csv(f'{save_dir}/y_test.csv')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(2469246, 21) (2469246, 1) (611291, 21) (611291, 1)


### (2) Lựa chọn các đặc trưng

In [4]:
x_train

,amt,gender,city_pop,age,trans_month,trans_year,latitudinal_distance,longitudinal_distance,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,-0.000093,0,-0.293456,0.620236,4,1,-0.426654,-0.446743,False,False,False,False,False,False,False,False,False,True,False,False,False
1,-0.077031,1,0.089542,-1.090464,7,1,0.862002,1.717500,False,False,False,False,False,False,True,False,False,False,False,False,False
2,-0.044826,1,-0.226449,-1.381506,7,0,0.775399,-1.014640,False,False,False,False,True,False,False,False,False,False,False,False,False
3,0.337968,1,-0.278547,-0.512469,7,0,1.381621,0.145394,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0.962903,1,-0.287644,0.239836,7,1,0.584872,0.443194,False,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469241,2.599739,0,1.735005,-0.250147,4,0,-0.338516,-0.290912,False,False,False,True,False,False,False,False,False,False,False,False,False
2469242,5.381776,1,0.391923,0.319283,8,1,0.763022,-0.259325,False,False,False,False,False,False,False,False,False,False,True,False,False
2469243,5.027570,1,-0.293713,0.399183,0,0,-1.228683,-0.184009,False,False,False,False,False,False,False,False,False,False,True,False,False
2469244,1.541057,1,-0.291265,-1.473200,7,0,-0.141900,-0.076251,False,False,False,True,False,False,False,False,False,False,False,False,False


In [5]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [6]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo Empty DataFrame
Columns: [Feature, Coefficient]
Index: []
Xếp hạng các đặc trưng theo mức độ quan trọng:
                    Feature  Coefficient
9    category_gas_transport     4.857507
17   category_personal_care     3.974522
10     category_grocery_net     3.810897
11     category_grocery_pos     3.786735
14       category_kids_pets     3.730138
16        category_misc_pos     3.680412
20          category_travel     3.466767
12  category_health_fitness     3.346476
8      category_food_dining     3.059403
13            category_home     2.667064
0                       amt     1.871810
18    category_shopping_net    -0.817439
15        category_misc_net     0.707122
5                trans_year    -0.608860
1                    gender    -0.577923
19    category_shopping_pos    -0.489375
2                  city_pop    -0.145159
4               trans_month    -0.081706
7     longitudinal_distance    -0.025231
6      latitudinal_distance     0.018903
3                   

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [7]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [8]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['category_food_dining', 'category_gas_transport', 'category_grocery_net', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_pos', 'category_personal_care', 'category_travel']
RFE
Kiểm tra mô hình LDA
['amt', 'category_food_dining', 'category_gas_transport', 'category_grocery_net', 'category_grocery_pos', 'category_health_fitness', 'category_kids_pets', 'category_misc_pos', 'category_personal_care', 'category_travel']
RFE
Kiểm tra mô hình DTC
['age', 'amt', 'category_gas_transport', 'category_grocery_net', 'category_home', 'city_pop', 'gender', 'latitudinal_distance', 'longitudinal_distance', 'trans_month']
RFE
Kiểm tra mô hình RD
['age', 'amt', 'category_gas_transport', 'category_grocery_pos', 'category_misc_net', 'category_shopping_net', 'city_pop', 'latitudinal_distance', 'longitudinal_distance', 'trans_month']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [Info] Number of positive: 1234623, number of negative: 12

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [9]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], dtype='object')
Lua chon dac trung GNB
Index(['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], dtype='object')
Lua chon dac trung NN
Index(['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], dtype='object')
Lua chon dac trung BG
Index(['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], dtype='object')
Lu

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [10]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], 'GNB': ['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], 'NN': ['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], 'BG': ['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_net', 'category_shopping_net'], 'NB': ['amt', 'age', 'trans_month', 'trans_year', 'category_food_dining', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_k

In [11]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['category_food_dining',
  'category_gas_transport',
  'category_grocery_net',
  'category_grocery_pos',
  'category_health_fitness',
  'category_home',
  'category_kids_pets',
  'category_misc_pos',
  'category_personal_care',
  'category_travel'],
 'LDA': ['amt',
  'category_food_dining',
  'category_gas_transport',
  'category_grocery_net',
  'category_grocery_pos',
  'category_health_fitness',
  'category_kids_pets',
  'category_misc_pos',
  'category_personal_care',
  'category_travel'],
 'DTC': ['age',
  'amt',
  'category_gas_transport',
  'category_grocery_net',
  'category_home',
  'city_pop',
  'gender',
  'latitudinal_distance',
  'longitudinal_distance',
  'trans_month'],
 'RD': ['age',
  'amt',
  'category_food_dining',
  'category_gas_transport',
  'category_grocery_pos',
  'category_shopping_net',
  'city_pop',
  'latitudinal_distance',
  'longitudinal_distance',
  'trans_month'],
 'LGBM': ['age',
  'amt',
  'category_food_dining',
  'category_gas_transport',
  'c

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [12]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['category_food_dining', 'category_gas_transport', 'category_grocery_net', 'category_grocery_pos', 'category_health_fitness', 'category_home', 'category_kids_pets', 'category_misc_pos', 'category_personal_care', 'category_travel'], 'LDA': ['amt', 'category_food_dining', 'category_gas_transport', 'category_grocery_net', 'category_grocery_pos', 'category_health_fitness', 'category_kids_pets', 'category_misc_pos', 'category_personal_care', 'category_travel'], 'DTC': ['age', 'amt', 'category_gas_transport', 'category_grocery_net', 'category_home', 'city_pop', 'gender', 'latitudinal_distance', 'longitudinal_distance', 'trans_month'], 'RD': ['age', 'amt', 'category_food_dining', 'category_gas_transport', 'category_grocery_pos', 'category_shopping_net', 'city_pop', 'latitudinal_distance', 'longitudinal_distance', 'trans_month'], 'LGBM': ['age', 'amt', 'category_food_dining', 'category_gas_transport', 'category_home', 'category_misc_pos', 'category_travel', 'city_pop', 'latitudina